# Summary of logic of the strategy.

I have the model, I know the reactions that are needed.
1.	Add the reactions
2.	Does it grow on the c source? MAX GROWTH
3.	Does it produce what wanted? If growth is null, add lower bound = 5% MAX GROWTH
4.	Add free balancing reactions
5.	Remove maintenance
6.	Set constraints as for right stoichiometry
7.	FBA with objective = production
8.	pFBA

Step 4 and 5 are needed in order for step 7 to give result, because balancing reactions are included even if biomass is not being created. 
Setting the reactions to irreversible might be done before step 7, but then the constraints must be checked carefully. this step is avoided in here as well as in the pipeline's workflow. 

In [1]:
import cobra

from cobra.flux_analysis import phenotype_phase_plane
from cobra.flux_analysis import production_envelope
from cobra.flux_analysis import gapfill, pfba
from cobra.manipulation import modify 
import csv
from pipeline.scripts.input_parser import *
from pipeline.scripts.import_models import get_universal_main
from pipeline.scripts.analysis import *
import cobra.test
import os
from os.path import join

DEBUG:matplotlib:(private) matplotlib data path: C:\Users\delip\anaconda3\envs\cameo\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:matplotlib data path: C:\Users\delip\anaconda3\envs\cameo\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\delip\.matplotlib
DEBUG:matplotlib:matplotlib version 3.3.0
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'errno', 'stat', '_stat', 'ntpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keywo

DEBUG:matplotlib:CACHEDIR=C:\Users\delip\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\delip\.matplotlib\fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
ecoli = cobra.Model("reference_model")
ecoli = cobra.io.read_sbml_model('iML1515.xml')

INFO:gurobipy.gurobipy:


--------------------------------------------


INFO:gurobipy.gurobipy:--------------------------------------------


INFO:gurobipy.gurobipy:Warning: your license will expire in 9 days


--------------------------------------------


...

INFO:gurobipy.gurobipy:


Using license file C:\Users\delip\gurobi.lic


INFO:gurobipy.gurobipy:Using license file C:\Users\delip\gurobi.lic


Academic license - for non-commercial use only


INFO:gurobipy.gurobipy:Academic license - for non-commercial use only


In [3]:
shut_down_c_sources('pipeline/inputs/new_ch4_for.csv', ecoli)


---Carbon sources---


In [3]:
universal = get_universal_main('pipeline/inputs/','pipeline/inputs/new_ch4_for.csv')

cobra\core\model.py:716 UserWarning: need to pass in a list


## 1. Add reactions needed for Pathway

In [4]:
main('pipeline/inputs/new_ch4_for.csv', universal, ecoli)


ch4 is in the medium 

Exchange ch4:  ch4_e -->  Old bounds:  (0.0, 1000.0)

nh4 is in the medium 

Exchange nh4:  nh4_e <=>  Old bounds:  (-1000.0, 1000.0)

for is in the medium 

Exchange for:  for_e -->  Old bounds:  (0.0, 1000.0)
For ch4 there isn't any uptake trasnsporter in the reference model
The trasporter has been added to the 
                                reference model from the input file
For nh4 there isn't any uptake trasnsporter in the reference model
For for there isn't any uptake trasnsporter in the reference model

The following reactions will be added to the universal model: ['R01142', 'R01143', 'MMO1', 'MMO2']
unknown metabolite 'ch4_c' created
unknown metabolite 'focytcc_c' created
unknown metabolite 'ficytcc_c' created


Name,iML1515
Memory address,0x015eef91ca58
Number of metabolites,1877
Number of reactions,2713
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
ecoli.reactions.EX_ch4_e

Reaction identifier,EX_ch4_e
Name,EX ch4 LPAREN e RPAREN
Memory address,0x015ef880bf28
Stoichiometry,ch4_e <-- Methane <--
GPR,
Lower bound,-1000
Upper bound,0


In [6]:
mmo = universal.reactions.MMO2
ecoli.add_reaction(mmo)

In [7]:
comp = ecoli.metabolites.q8h2_c # avoid exception related to missing KEGG identifiers
comp.annotation['kegg.compound'] = 'C17569'

In [8]:
alcd1 = universal.reactions.ALCD1
ecoli.add_reaction(alcd1)

In [9]:
ecoli.reactions.get_by_id('MMO2')

Reaction identifier,MMO2
Name,Methane monooxygenase quinol 8
Memory address,0x015efbc00128
Stoichiometry,ch4_c + o2_c + q8h2_c <=> h2o_c + meoh_c + q8_c Methane + O2 O2 + Ubiquinol-8 <=> H2O H2O + Methanol + Ubiquinone-8
GPR,( MEALZ_0514 and MEALZ_0515 and MEALZ_0516 ) or 6666666.378008.peg.3726 or 6666666.378008.peg.397...
Lower bound,-1000
Upper bound,1000


In [10]:
alcd1

Reaction identifier,ALCD1
Name,Alcohol dehydrogenase (methanol)
Memory address,0x015efacc2710
Stoichiometry,meoh_c + nad_c <=> fald_c + h_c + nadh_c Methanol + Nicotinamide adenine dinucleotide <=> Formaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## 2. Does it grows on CH4?

In [11]:
glc = ecoli.reactions.EX_glc__D_e
glc.bounds = (0,0)

In [12]:
print(ecoli.objective.expression)

1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685


In [13]:
cobra.flux_analysis.phenotype_phase_plane.find_carbon_sources(ecoli)

[]

In [14]:
metex = ecoli.reactions.EX_ch4_e
metex.bounds 

(-1000, 0)

In [15]:
biomass = ecoli.reactions.BIOMASS_Ec_iML1515_core_75p37M
biomass.upper_bound = 0.87

In [16]:
co2ex = ecoli.reactions.EX_co2_e
co2ex.lower_bound = 0

In [17]:
fba_check = ecoli.optimize()
print(fba_check.objective_value, '\n', fba_check.fluxes['EX_ch4_e'])

0.87 
 -599.097257901152


It does consume as much methane as possible!

## 3. Does it produce what I want? 

In [18]:
fba_check.fluxes['EX_for_e']

185.46106704230345

##### not while optimizing growth
What if I put a constraint for its production?

Set lb = 1

In [19]:
forex = ecoli.reactions.get_by_id('EX_for_e')
forex.lower_bound = 1

In [20]:
fba_check_production = ecoli.optimize()
print(fba_check_production.objective_value, '\n', fba_check_production.fluxes['EX_ch4_e'])

0.87 
 -599.0972579011524


In [21]:
fba_check_production.fluxes['EX_for_e']

185.46106704230505

## 4. Add free balancing reactions

In [22]:
ATPM_revert = cobra.Reaction('ATPM_revert')
ecoli.add_reaction(ATPM_revert)
atpmr = 'adp_c + h_c + pi_c --> atp_c + h2o_c'
ATPM_revert.build_reaction_from_string(atpmr, fwd_arrow='-->')

In [23]:
endless_nadph = cobra.Reaction('endless_nadph')
ecoli.add_reaction(endless_nadph)
new_nadph = 'nadp_c + h_c --> nadph_c'
endless_nadph.build_reaction_from_string(new_nadph, fwd_arrow='-->')

In [24]:
endless_nadh = cobra.Reaction('endless_nadh')
ecoli.add_reaction(endless_nadh)
new_nadh = 'nad_c + h_c --> nadh_c'
endless_nadh.build_reaction_from_string(new_nadh, fwd_arrow='-->')

In [25]:
endless_fad2h = cobra.Reaction('endless_fad2h')
ecoli.add_reaction(endless_fad2h)
new_fad2h = 'fad_c + h_c --> fadh2_c'
endless_fad2h.build_reaction_from_string(new_fad2h, fwd_arrow='-->')

In [26]:
endless_nad = cobra.Reaction('endless_nad')
ecoli.add_reaction(endless_nad)
new_nad = 'nadh_c --> nad_c + h_c'
endless_nad.build_reaction_from_string(new_nad, fwd_arrow='-->')

In [27]:
endless_q8h2 = cobra.Reaction('endless_q8h2')
ecoli.add_reaction(endless_q8h2)
new_q8h2 = 'q8_c + h_c --> q8h2_c'
endless_q8h2.build_reaction_from_string(new_q8h2, fwd_arrow='-->')

In [28]:
endless_coa2 = cobra.Reaction('endless_coa2')
ecoli.add_reaction(endless_coa2)
new_coa2 = 'accoa_c --> ac_c + coa_c'
endless_coa2.build_reaction_from_string(new_coa2, fwd_arrow='-->')

In [29]:
endless_coa2

Reaction identifier,endless_coa2
Name,
Memory address,0x015efbc5ff60
Stoichiometry,accoa_c --> ac_c + coa_c Acetyl-CoA --> Acetate + Coenzyme A
GPR,
Lower bound,0
Upper bound,1000.0


## 5. Remove maintenance

In [30]:
for i in ecoli.reactions:
    if 'maintenance' in i.name:
        print(i.id, i.name)
        ecoli.remove_reactions([i.id])

ATPM ATP maintenance requirement


## 6. Set constraints for right stoichiometry

- CH4 to -4
- FOR to 2

In [31]:
metex.lower_bound = (-1.1)
metex.upper_bound = (-0.9)
print(metex.bounds)

(-1.1, -0.9)


In [32]:
ecoli.reactions.EX_co2_e

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x015ef8197cf8
Stoichiometry,co2_e --> CO2 CO2 -->
GPR,
Lower bound,0
Upper bound,1000.0


In [33]:
forex.lower_bound = 1

In [34]:
co2ex = ecoli.reactions.EX_co2_e
co2ex.lower_bound = 0

## 7. FBA with objective = production

In [35]:
ecoli.objective = forex

#### Oxygen sentsitivity

In [36]:
print(len(ecoli.reactions))
for i in ecoli.reactions:
    if 'flx' in i.reaction:
        print(i.id, i.name, i.reaction)
        ecoli.remove_reactions([i.id])
print(len(ecoli.reactions))

2721
POR5 Pyruvate synthase coa_c + 2.0 flxso_c + pyr_c <=> accoa_c + co2_c + 2.0 flxr_c + h_c
MECDPDH5 2C-methyl-D-erythritol 2,4 cyclodiphosphate dehydratase 2mecdp_c + 2.0 flxr_c + h_c --> 2.0 flxso_c + h2mb4p_c + h2o_c
RNTR1c2 Ribonucleoside-triphosphate reductase (ATP) (flavodoxin) atp_c + 2.0 flxr_c + 2.0 h_c --> datp_c + 2.0 flxso_c + h2o_c
RNTR3c2 Ribonucleoside-triphosphate reductase (CTP) (flavodoxin) ctp_c + 2.0 flxr_c + 2.0 h_c --> dctp_c + 2.0 flxso_c + h2o_c
FLDR2 Flavodoxin reductase (NADPH) 2.0 flxso_c + nadph_c --> 2.0 flxr_c + h_c + nadp_c
2716


In [37]:
pfl = ecoli.reactions.get_by_id('PFL')
pfl.bounds = (0,0)

In [38]:
for i in ecoli.reactions:
    if 'trd' in i.reaction:
        print(i.id, i.name, i.reaction)
        ecoli.remove_reactions([i.id])
print(len(ecoli.reactions))

RNDR1 Ribonucleoside-diphosphate reductase (ADP) adp_c + trdrd_c --> dadp_c + h2o_c + trdox_c
RNDR3 Ribonucleoside-diphosphate reductase (CDP) cdp_c + trdrd_c --> dcdp_c + h2o_c + trdox_c
TRDR Thioredoxin reductase (NADPH) h_c + nadph_c + trdox_c --> nadp_c + trdrd_c
PAPSR Phosphoadenylyl-sulfate reductase (thioredoxin) paps_c + trdrd_c --> 2.0 h_c + pap_c + so3_c + trdox_c
METSOXR1 L-methionine-S-oxide reductase metsox_S__L_c + trdrd_c --> h2o_c + met__L_c + trdox_c
RNDR2 Ribonucleoside-diphosphate reductase (GDP) gdp_c + trdrd_c --> dgdp_c + h2o_c + trdox_c
THIORDXi Hydrogen peroxide reductase (thioredoxin) h2o2_c + trdrd_c --> 2.0 h2o_c + trdox_c
DSBDR DsbD reductase dsbdox_c + trdrd_c --> dsbdrd_c + trdox_c
2708


In [39]:
fba_production = ecoli.optimize()
print(fba_production.objective_value, '\n', fba_production.fluxes['EX_ch4_e'], '\n', fba_production.fluxes['EX_for_e'])

1.1 
 -1.1 
 1.1


In [40]:
fba_production.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.0

## 8. pFBA 

In [41]:
pfba_solution = pfba(ecoli)

In [42]:
print(pfba_solution.fluxes['EX_co2_e'])

0.0


In [43]:
pfba_solution.fluxes['EX_ch4_e']

-1.1

In [44]:
print(pfba_solution.fluxes['EX_for_e'])

1.1


In [45]:
pfba_solution.objective_value

23.10000000000001

In [46]:
non_zero = []
for i in pfba_solution.fluxes:
    if i != 0:
        non_zero.append(i)

len(non_zero)

17

In [47]:
sol_list = []
for i in ecoli.reactions:
    if pfba_solution.fluxes[i.id] > 1e-10 or pfba_solution.fluxes[i.id] < -1e-10:
        sol_list.append(i)
print(len(sol_list))

17


In [48]:
for i in sol_list:
    print('\n',i.id, i.name, i.reaction, pfba_solution.fluxes[i.id])
    
print(len(sol_list))


 EX_h_e H+ exchange h_e <=>  3.3000000000000007

 EX_for_e Formate exchange for_e -->  1.1

 Htex Proton transport via diffusion (extracellular to periplasm) h_e <=> h_p -3.3000000000000007

 O2tpp O2 transport via diffusion (periplasm) o2_p <=> o2_c 1.1

 EX_ch4_e EX ch4 LPAREN e RPAREN  ch4_e <--  -1.1

 O2tex Oxygen transport via diffusion (extracellular to periplasm) o2_e <=> o2_p 1.1

 FORtppi Formate transport via diffusion (cytoplasm to periplasm) for_c --> for_p 1.1

 FALDH2 Formaldehyde dehydrogenase hmgth_c + nad_c <=> Sfglutth_c + h_c + nadh_c 1.1

 FALGTHLs Formaldehyde glutathione ligase (spontaneous) fald_c + gthrd_c <=> hmgth_c 1.1

 FORtex Formate transport via diffusion (extracellular to periplasm) for_e <=> for_p -1.1

 SFGTHi S-Formylglutathione hydralase Sfglutth_c + h2o_c --> for_c + gthrd_c + h_c 1.1

 EX_o2_e O2 exchange o2_e <=>  -1.1

 NADH16pp NADH dehydrogenase (ubiquinone-8 & 3 protons) (periplasm) 4.0 h_c + nadh_c + q8_c --> 3.0 h_p + nad_c + q8h2_c 1.1000

## Shortening the list

Remove the exchanges and transport reactions to get to the minimal list of reactions to do MDF on

In [49]:
to_exclude_id = ['redp', 'THD2pp', 'CYTBO3_4pp', 'NADH16pp', 'EX_', 'endless', 'tex', 'tpp', 'ATPS4rpp', 'ATPM_revert', 'DM_', 'NTPTP']
to_exclude_name = ['ranspor', 'equilibr', 'anaer', 'maintenance' ]

In [50]:
for ele in to_exclude_id:
    for i in list(sol_list):
        if ele in i.id:
            print(i.id, i.name, i.reaction, pfba_solution.fluxes[i.id])
            sol_list.remove(i)
            
print(len(sol_list))

NADH16pp NADH dehydrogenase (ubiquinone-8 & 3 protons) (periplasm) 4.0 h_c + nadh_c + q8_c --> 3.0 h_p + nad_c + q8h2_c 1.1000000000000003
EX_h_e H+ exchange h_e <=>  3.3000000000000007
EX_for_e Formate exchange for_e -->  1.1
EX_ch4_e EX ch4 LPAREN e RPAREN  ch4_e <--  -1.1
EX_o2_e O2 exchange o2_e <=>  -1.1
endless_nad  nadh_c --> h_c + nad_c 1.0999999999999999
Htex Proton transport via diffusion (extracellular to periplasm) h_e <=> h_p -3.3000000000000007
O2tex Oxygen transport via diffusion (extracellular to periplasm) o2_e <=> o2_p 1.1
FORtex Formate transport via diffusion (extracellular to periplasm) for_e <=> for_p -1.1
O2tpp O2 transport via diffusion (periplasm) o2_p <=> o2_c 1.1
FORtppi Formate transport via diffusion (cytoplasm to periplasm) for_c --> for_p 1.1
6


In [51]:
for ele in to_exclude_name:
    for i in list(sol_list):
        if ele in i.name:
            print(i.id, i.name, i.reaction, pfba_solution.fluxes[i.id])
            sol_list.remove(i)
            
print(len(sol_list))

CH4t CH4_ transporter ch4_e --> ch4_c 1.1
5


In [52]:
for i in sol_list:
    print('\n',i.id, i.name, i.reaction, pfba_solution.fluxes[i.id])
    
print(len(sol_list))


 FALDH2 Formaldehyde dehydrogenase hmgth_c + nad_c <=> Sfglutth_c + h_c + nadh_c 1.1

 FALGTHLs Formaldehyde glutathione ligase (spontaneous) fald_c + gthrd_c <=> hmgth_c 1.1

 SFGTHi S-Formylglutathione hydralase Sfglutth_c + h2o_c --> for_c + gthrd_c + h_c 1.1

 MMO2 Methane monooxygenase quinol 8 ch4_c + o2_c + q8h2_c <=> h2o_c + meoh_c + q8_c 1.1

 ALCD1 Alcohol dehydrogenase (methanol) meoh_c + nad_c <=> fald_c + h_c + nadh_c 1.1
5


# Manual MDF analysis
### SB-tab generation for MDF

In [53]:
import sbtab
from sbtab import SBtab, validatorSBtab, sbml2sbtab

from equilibrator_api import ComponentContribution, Q_
from equilibrator_pathway.pathway import Pathway
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd

In [54]:
compounds1 = []
for i in sol_list:
    for k in i.metabolites:
        if k not in compounds1:
            compounds1.append(k)

In [55]:
print(len(compounds1))
print(len(sol_list))
for metab in compounds1:
    if 'kegg.compound' not in metab.annotation.keys():
        compounds1.remove(metab)
        print('This coupound {} does not exist in equilibrator cache'.format(metab.id))
        for reaction in metab.reactions:
            for esse in list(sol_list):
                if reaction.id == esse.id:
                    sol_list.remove(esse)
        
        
print(len(compounds1))
print(len(sol_list))

14
5
14
5


In [56]:
print(len(compounds1))
print(len(sol_list))
for metab in compounds1:
    if 'inchi_key' not in metab.annotation.keys():
        compounds1.remove(metab)
        print('This coupound {} does not have inchi_key'.format(metab.id))
        for reaction in metab.reactions:
            for esse in list(sol_list):
                if reaction.id == esse.id:
                    sol_list.remove(esse)
        
        
print(len(compounds1))
print(len(sol_list))

14
5
14
5


In [57]:
print(len(sol_list))

for i in list(sol_list):
    out = i.check_mass_balance()
    if out != {}:
        print(i.id, i.reaction)
        #sol_list.remove(i)

            
print(len(sol_list))

5
5


In [58]:
print(len(compounds1))
new_comp1 = []
for i in sol_list:
    for k in i.metabolites:
        if k not in new_comp1:
            new_comp1.append(k)
print(len(new_comp1))

14
14


In [59]:
ecoli.optimize()

,fluxes,reduced_costs
CYTDK2,0.000,0.0
XPPT,0.000,0.0
HXPRT,0.000,0.0
NDPK5,0.000,0.0
SHK3Dr,0.000,0.0
...,...,...
endless_nadh,0.000,0.0
endless_fad2h,0.000,0.0
endless_nad,0.275,0.0
endless_q8h2,0.000,0.0


In [60]:
essential_dict1={}
for i in sol_list:
    essential_dict1[i.id] = i.flux

In [61]:
with open('ch4_for_04.tsv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    #SBtab Reaction
    writer.writerow(["!!SBtab", "TableID='Reaction'", "TableType='Reaction'"])
    writer.writerow(["!ID", "!ReactionFormula"])
    for i in sol_list:
        react = i.reaction
        react = i.reaction
        #refined_react = remove_c(react)

        if ' --> ' in react:
                    new_react = react.replace(' --> ', ' <=> ')
                    writer.writerow([i.id.lower(), new_react])
        elif ' <-- ' in react:
                    new_react = react.replace(' <-- ', ' <=> ')
                    writer.writerow([i.id.lower(), new_react])
        elif ' <=> ' in react:
                    writer.writerow([i.id.lower(), react])
    
    writer.writerow([])
    
    #SBtab Compounds
    writer.writerow(["!!SBtab", "TableID='Compound'", "TableType='Compound'"])
    writer.writerow(["!ID", "!Identifiers"])
    for i in new_comp1:
        
        if 'kegg.compound' in i.annotation.keys():
            identifier = i.annotation['kegg.compound']
            if type(identifier)!=list:
                writer.writerow([i.id, "kegg:{}".format(identifier)])
            else:
                writer.writerow([i.id, "kegg:{}".format(identifier[0])])
        else:
             "MissingKEGG_ID: Compound {} does not have any assigned KEGG Identifier".format(i)
    
    writer.writerow([])
    
    #SBtab Flux
    writer.writerow(["!!SBtab", "TableID='Flux'", "TableType='Quantity'", ":Unit='mM/s'"])
    writer.writerow(["!QuantityType", "!Reaction", "!Value"])
    for key, val in essential_dict1.items():
        writer.writerow(['rate of reaction', key.lower(), val])
    
    writer.writerow([])
    
    #SBtab ConcentrationConstraint
    writer.writerow(["!!SBtab", "TableID='ConcentrationConstraint'", "TableType='Quantity'", ":Unit='mM'"])
    writer.writerow(["!QuantityType", "!Compound", "!Min", "!Max"])
    
    for i in new_comp1:
        if i.id == 'ch4_c':
            writer.writerow(['concentration', i.id, 1.247, 8.105])
        elif i.id == 'pi_c':
            writer.writerow(['concentration', i.id, 10, 10])
        elif i.id == 'ppi_c':
            writer.writerow(['concentration', i.id, 1, 1])
        elif i.id == 'coa_c':
            writer.writerow(['concentration', i.id, 0.1, 5])
        elif i.id == 'atp_c':
            writer.writerow(['concentration', i.id, 5, 5])
        elif i.id == 'adp_c':
            writer.writerow(['concentration', i.id, 0.5, 0.5])
        elif i.id == 'amp_c':
            writer.writerow(['concentration', i.id, 0.5, 0.5])
        else:
            writer.writerow(['concentration', i.id, 0.001, 10])

In [62]:
Sd_config = SBtab.read_csv('configuration.tsv', 'Sd_config', xlsx=False)
tab_config = Sd_config.get_sbtab_by_id('Configuration')



# create an SBtab Document Object Sd
Sd = SBtab.SBtabDocument('CH4_FOR4_mmo26')
Sd.set_name('CH4_FOR4_mmo26.tsv')
Sd.add_sbtab(tab_config)

    
# you can add further SBtab tables or strings to the document
# by using the functions add_sbtab() or add_sbtab_string()

Sd_full1 = SBtab.read_csv('ch4_for_04.tsv', 'CO2_PRY', xlsx=False)

tab_rea = Sd_full1.get_sbtab_by_id('Reaction')
Sd.add_sbtab(tab_rea)

tab_com = Sd_full1.get_sbtab_by_id('Compound')
Sd.add_sbtab(tab_com)

tab_flux = Sd_full1.get_sbtab_by_id('Flux')
Sd.add_sbtab(tab_flux)

tab_cc = Sd_full1.get_sbtab_by_id('ConcentrationConstraint')
Sd.add_sbtab(tab_cc)

Sd.write(filename='CH4_FOR4.tsv')

DEBUG:root:name = Reaction_unnamed, type = Reaction
DEBUG:root:name = Compound_unnamed, type = Compound
DEBUG:root:name = Quantity_unnamed, type = Quantity
DEBUG:root:name = Quantity_unnamed, type = Quantity


True

In [63]:
comp_contrib = ComponentContribution()

INFO:equilibrator_cache.api:Fetching eQuilibrator compound cache...
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): pkg.quiltdata.com:443


DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/version/equilibrator/cache/0.2.10 HTTP/1.1" 200 237
DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/package/equilibrator/cache/d11d9ebd1cf25d95d8c1369a8ca2bbe46eb734f8b2b9a4225f16de614104e12c?subpath=&meta_only= HTTP/1.1" 200 None
DEBUG:equilibrator_cache.compound_cache:Loading synonyms into data frame.


Fragments already downloaded


INFO:component_contribution.parameters:Fetching Component-Contribution parameters...


DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/version/equilibrator/component_contribution/0.2.18 HTTP/1.1" 200 237
DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/package/equilibrator/component_contribution/2c14d76366ba17dc876f2fc10612db41cae65c0411d8a654b762faa2beb3b055?subpath=&meta_only= HTTP/1.1" 200 None


Fragments already downloaded


In [64]:
pp = Pathway.from_sbtab("CH4_FOR4.tsv", comp_contrib=comp_contrib)
pp.update_standard_dgs()

DEBUG:root:name = Config_unnamed, type = Config
DEBUG:root:name = Reaction_unnamed, type = Reaction
DEBUG:root:name = Compound_unnamed, type = Compound
DEBUG:root:name = Quantity_unnamed, type = Quantity
DEBUG:root:name = Quantity_unnamed, type = Quantity
DEBUG:root:Looking for C14180 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00003 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C01031 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00080 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00004 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00067 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00051 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00001 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00058 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C01438 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00007 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C17569 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00132 in keg

In [65]:
mdf_sol = pp.calc_mdf()

In [66]:
mdf_sol.mdf

14.111393740986765 <Unit('kilojoule / mole')>

# Thermodynamic analysis with pipeline's functions

In [67]:
from pipeline.scripts.path_definition_mdf import whole_procedure_path_definition, get_mdf_value 

In [68]:
pruned_path = whole_procedure_path_definition('new_ch4_for.csv', ecoli, '{}_Run_{}.tsv'.format('for2', '02-11_newch4'), 'input_mdf_{}_Run_{}.tsv'.format('for2', '02-11_newch4'))

unknown metabolite 'qh2_c' created
Substrate's exchange reaction bounds :  (-1.1, -0.9)
Target's exchange reaction bounds :  (1.0, 1000)
Carbon source:  []
Target:  EX_for_e: for_e --> 
FBA objective value:  1.1 
Substrate consumption flux:  -1.1 
Target production flux:  1.1 
Biomass:  0.0 

pFBA objective value:  23.10000000000001 
Substrate consumption flux:  -1.1 
Target production flux:  1.1 
Biomass:  0.0 

Raw list length =  17
Pruned list length =  5


DEBUG:root:name = Reaction_unnamed, type = Reaction
DEBUG:root:name = Compound_unnamed, type = Compound
DEBUG:root:name = Quantity_unnamed, type = Quantity
DEBUG:root:name = Quantity_unnamed, type = Quantity


In [69]:
if pruned_path != None and pruned_path != 0:
        pathways_MDF = get_mdf_value("input_mdf_for2_Run_02-11_newch4.tsv")
        path_len = len(pruned_path)
else:
        print('The thermodynimac analysis has been unsuccesful')
        pathways_MDF = None
        path_len = None

INFO:equilibrator_cache.api:Fetching eQuilibrator compound cache...
DEBUG:urllib3.connectionpool:Resetting dropped connection: pkg.quiltdata.com


DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/version/equilibrator/cache/0.2.10 HTTP/1.1" 200 237
DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/package/equilibrator/cache/d11d9ebd1cf25d95d8c1369a8ca2bbe46eb734f8b2b9a4225f16de614104e12c?subpath=&meta_only= HTTP/1.1" 200 None
DEBUG:equilibrator_cache.compound_cache:Loading synonyms into data frame.


Fragments already downloaded


INFO:component_contribution.parameters:Fetching Component-Contribution parameters...
DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/version/equilibrator/component_contribution/0.2.18 HTTP/1.1" 200 237


DEBUG:urllib3.connectionpool:https://pkg.quiltdata.com:443 "GET /api/package/equilibrator/component_contribution/2c14d76366ba17dc876f2fc10612db41cae65c0411d8a654b762faa2beb3b055?subpath=&meta_only= HTTP/1.1" 200 None


Fragments already downloaded


DEBUG:root:name = Config_unnamed, type = Config
DEBUG:root:name = Reaction_unnamed, type = Reaction
DEBUG:root:name = Compound_unnamed, type = Compound
DEBUG:root:name = Quantity_unnamed, type = Quantity
DEBUG:root:name = Quantity_unnamed, type = Quantity
DEBUG:root:Looking for C14180 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00003 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C01031 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00080 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00004 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00067 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00051 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00001 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00058 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C01438 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00007 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C17569 in kegg
DEBUG:root:Cache hit
DEBUG:root:Looking for C00132 in keg

In [70]:
pathways_MDF

14.111393740986765 <Unit('kilojoule / mole')>

In [71]:
path_len

5